# villaInventorySdk
> read and write inventory in real time

## Install

`pip install villaInventorySdk`

## How to use

Uploading a large amount of data

In [1]:
#hide
isAws = True
if isAws:
  USER = None
  PW = None
else:
  import pickle
  KEY = ''
  PW = ''
  keypath = '/Users/nic/.villa-product-master-inventory'
  if KEY and PW:
    with open (keypath, 'wb') as f:
      pickle.dump({
          'KEY': KEY,
          'PW': PW
      }, f)
  with open(keypath, 'rb') as f:
    creden = pickle.load(f)
  USER = creden['KEY']
  PW = creden['PW']

## sample input

In [2]:
sampleInput = [ 
  {'ib_brcode': '1023', 'ib_cf_qty': '835', 'ib_prcode': '84621', 'new_ib_vs_stock_cv': '839'},
  {'ib_brcode': '1022', 'ib_cf_qty': '24', 'ib_prcode': '12424', 'new_ib_vs_stock_cv': '21'}
]

## Upload data

In [3]:
#hide
from villaInventorySdk.inventory import InventorySdk
from random import randrange
import boto3, time, json
from dataclasses import dataclass
from dataclasses_json import dataclass_json
from datetime import datetime

In [11]:
#hide
inputBucketName= 'input-bucket-dev-manual'
functionName= 'update-inventory-s3-dev-manual'
invocationType = 'RequestResponse'
branch = 'dev-manual'

In [12]:
#hide
inputBucketName= 'input-bucket-test-inventory-6b3753e2f2'
functionName= 'update-inventory-s3-test-inventory-6b3753e2f2'
invocationType = 'RequestResponse'

In [13]:
#hide
def timeit(method):
    def timed(*args, **kw):
        ts = time.time()
        result = method(*args, **kw)
        te = time.time()
        if 'log_time' in kw:
            name = kw.get('log_name', method.__name__.upper())
            kw['log_time'][name] = int((te - ts) * 1000)
        else:
            print ('%r took %2.2f ms' % \
                  (method.__name__, (te - ts) * 1000))
        return result
    return timed

## init sdk

In [23]:
%%time
sdk = InventorySdk(user=USER, pw=PW, branchName = branch)

CPU times: user 5.63 ms, sys: 103 µs, total: 5.73 ms
Wall time: 5.38 ms


## Update inventory 

In [28]:
%%time
sdk.updateWithS3( sampleInput )

CPU times: user 13 ms, sys: 8.05 ms, total: 21.1 ms
Wall time: 130 ms


{'body': 'true', 'statusCode': 200, 'header': {}}

## Query single product

In [29]:
%%time
sdk.querySingleProduct('0000002')

CPU times: user 13.7 ms, sys: 0 ns, total: 13.7 ms
Wall time: 64.3 ms


{'body': '{"ib_prcode":"0000002","1000":{"ib_cf_qty":35,"new_ib_bs_stock_cv":33,"lastUpdate":1600567810.529301},"1001":{"ib_cf_qty":32,"new_ib_bs_stock_cv":30,"lastUpdate":1600567810.529316},"1002":{"ib_cf_qty":34,"new_ib_bs_stock_cv":30,"lastUpdate":1600567810.529318},"lastUpdate":1600567810.529318}',
 'statusCode': 200,
 'header': {}}

## Query Branch

In [30]:
result = sdk.queryBranch('1000')
#showing the first 2 result
list(iter(result.items()))[:2]

AttributeError: 'InventorySdk' object has no attribute 'queryBranch'